Importing the libraries

In [4]:
import numpy as np

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
# from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install folium
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Reading the data

In [5]:
data = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
df=data[0]
df.head(3)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods


Applying the filters

In [6]:
dfWBorough = df[(df['Borough'] != "Not assigned")]

for neigh in dfWBorough['Neighbourhood']:
    if neigh == "Not assigned":
        neigh = dfWBorough['Borough']
dfWBorough = dfWBorough.groupby(['Postal Code','Borough'],sort=False).agg(lambda x: ','.join(x))

In [8]:
dfWBorough.reset_index(inplace=True)

In [9]:
dfWBorough.shape

(103, 4)

In [10]:
dfWBorough.head(3)

,index,Postal Code,Borough,Neighbourhood
0,0,M3A,North York,Parkwoods
1,1,M4A,North York,Victoria Village
2,2,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [11]:
# !pip install geopy
!pip install geocoder


     |████████████████████████████████| 98 kB 11.1 MB/s eta 0:00:01


In [12]:
import geocoder
latitude_arr = []
longitude_arr = []
for postal_code in dfWBorough['Postal Code']:
    lat_lng_coords = None
    while(lat_lng_coords is None):
        print("postal code ", postal_code)
        g=geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        print("here is location ", g.latlng)
        lat_lng_coords = g.latlng
    print("postal_code")
    latitude_arr.append(lat_lng_coords[0])
    longitude_arr.append(lat_lng_coords[1])
#     log = lat_lng_coords[1]
#     dfWBorough.append({'Latitude':latitude, 'Longitude':longitude}, ignore_index=True)
#     dfWBorough['Latitude'] = latitude
#     dfWBorough['Longitude'] = longitude
# dfWBorough 
print("done")

postal code  M3A
here is location  [43.75245000000007, -79.32990999999998]
postal_code
postal code  M4A
here is location  [43.73057000000006, -79.31305999999995]
postal_code
postal code  M5A
here is location  [43.65512000000007, -79.36263999999994]
postal_code
postal code  M6A
here is location  [43.72327000000007, -79.45041999999995]
postal_code
postal code  M7A
here is location  [43.66253000000006, -79.39187999999996]
postal_code
postal code  M9A
here is location  [43.662630000000036, -79.52830999999998]
postal_code
postal code  M1B
here is location  [43.811390000000074, -79.19661999999994]
postal_code
postal code  M3B
here is location  [43.74923000000007, -79.36185999999998]
postal_code
postal code  M4B
here is location  [43.70718000000005, -79.31191999999999]
postal_code
postal code  M5B
here is location  [43.65739000000008, -79.37803999999994]
postal_code
postal code  M6B
here is location  [43.70687000000004, -79.44811999999996]
postal_code
postal code  M9B
here is location  [43.65

here is location  [43.71174000000008, -79.57940999999994]
postal_code
postal code  M1X
here is location  [43.83412000000004, -79.21667999999994]
postal_code
postal code  M4X
here is location  [43.667880000000025, -79.36648999999994]
postal_code
postal code  M5X
here is location  [43.64828000000006, -79.38145999999995]
postal_code
postal code  M8X
here is location  [43.65319000000005, -79.51112999999998]
postal_code
postal code  M4Y
here is location  [43.66659000000004, -79.38132999999993]
postal_code
postal code  M7Y
here is location  [43.648690000000045, -79.38543999999996]
postal_code
postal code  M8Y
here is location  [43.632780000000025, -79.48944999999998]
postal_code
postal code  M8Z
here is location  [43.62513000000007, -79.52680999999995]
postal_code
done


In [13]:
# dfWBorough.tail()
# longitude_arr
dfWBorough['Latitude'] = pd.Series(latitude_arr)
dfWBorough['Longitude'] = pd.Series(longitude_arr)

In [14]:
dfWBorough.head(2)

,index,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,0,M3A,North York,Parkwoods,43.75245,-79.32991
1,1,M4A,North York,Victoria Village,43.73057,-79.31306


Getting the Latitude and Longitude of Toronto

In [15]:
location = geocoder.arcgis('Toronto, Ontario')
Toronto_latitude = location.latlng[0]
Toronto_longitude = location.latlng[1]
print("latitude is {}, and longitude is {}".format(Toronto_latitude, Toronto_longitude))

latitude is 43.648690000000045, and longitude is -79.38543999999996


Displying a map of Toronto with the locations

In [16]:
# map_manhattan = folium.Map(location=[Toronto_latitude, longitude], zoom_start=11)
tor_map = folium.Map(location=[Toronto_latitude, Toronto_longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(dfWBorough['Latitude'], dfWBorough['Longitude'], dfWBorough['Neighbourhood']):
    label = folium.Popup(label, parse_html=False)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(tor_map)  
    
tor_map

# Getting the Foursquare credentials

In [17]:
CLIENT_ID = 'MLLMHYKAGPATS41JYT04LNU4KNMTB1SUZX5SPYVNGT2E2PN5' # your Foursquare ID
CLIENT_SECRET = '5SD4AM4YO3MKL0LQ2EPJU4JA30VLOOBXU1RQ1YV43MGU5DYD' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MLLMHYKAGPATS41JYT04LNU4KNMTB1SUZX5SPYVNGT2E2PN5
CLIENT_SECRET:5SD4AM4YO3MKL0LQ2EPJU4JA30VLOOBXU1RQ1YV43MGU5DYD


Creating a function to get the data from Foursquare api

In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
toronto_neighs = getNearbyVenues(dfWBorough['Neighbourhood'], dfWBorough['Latitude'], dfWBorough['Longitude'])

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [22]:
print(toronto_neighs.shape)
toronto_neighs.head(2)

(2404, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.75245,-79.32991,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.75245,-79.32991,Variety Store,43.751974,-79.333114,Food & Drink Shop


Now, we are going to explore the data

In [23]:
toronto_neighs.groupby('Neighbourhood').count().head(3)

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,16,16,16,16,16,16
"Alderwood, Long Branch",4,4,4,4,4,4
Bayview Village,4,4,4,4,4,4


In [24]:
print("there are {} unique categories in Torono".format({len(toronto_neighs['Venue Category'].unique())}))

there are {269} unique categories in Torono


New we are going to analyze each row

In [25]:
# one hot encoding
toronto_categs = pd.get_dummies(toronto_neighs[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_categs['Neighbourhood'] = toronto_neighs['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_categs.columns[-1]] + list(toronto_categs.columns[:-1])
# fixed_columns
toronto_categs = toronto_categs[fixed_columns]

toronto_categs.head(30)

,Neighbourhood,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bike Trail,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Newsagent,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Road,Rock Climbing Spot,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo Exhibit
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [26]:
toronto_categs.shape

(2404, 270)

In [31]:
toronto_categs = toronto_categs.groupby('Neighbourhood').mean().reset_index()
toronto_categs.head(4)

,Neighbourhood,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bike Trail,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Newsagent,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Road,Rock Climbing Spot,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo Exhibit
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0625,0.0,0.0625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

this is a function to sort the venues in descending order.

In [28]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [32]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_categs['Neighbourhood']

for ind in np.arange(toronto_categs.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_categs.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Hong Kong Restaurant,Discount Store,Skating Rink,Shopping Mall,Shanghai Restaurant,Supermarket,Sushi Restaurant,Bubble Tea Shop,Badminton Court
1,"Alderwood, Long Branch",Convenience Store,Pub,Performing Arts Venue,Gym,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Donut Shop
2,Bayview Village,Construction & Landscaping,Golf Driving Range,Trail,Park,Ethiopian Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Event Space
3,"Bedford Park, Lawrence Manor East",Sandwich Place,Italian Restaurant,Coffee Shop,Comfort Food Restaurant,Pub,Juice Bar,Restaurant,Thai Restaurant,Liquor Store,Indian Restaurant
4,Berczy Park,Coffee Shop,Cheese Shop,Cocktail Bar,Breakfast Spot,Beer Bar,Bakery,Restaurant,Seafood Restaurant,Farmers Market,Sandwich Place


Now we are going to cluster the Neighborhoods


In [34]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_categs.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 0, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

Now we are going to create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.


In [37]:
# add clustering labels
neighborhoods_venues_sorted.insert(1, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_neighs

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head(2) # check the last columns!

ValueError: cannot insert Cluster Labels, already exists

In [38]:
toronto_merged.head(2) # check the last columns!

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Parkwoods,43.75245,-79.32991,Brookbanks Park,43.751976,-79.332140,Park,2,Food & Drink Shop,Park,Falafel Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Farm,Dumpling Restaurant
1,Parkwoods,43.75245,-79.32991,Variety Store,43.751974,-79.333114,Food & Drink Shop,2,Food & Drink Shop,Park,Falafel Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Farm,Dumpling Restaurant


Now let us visulize the clusters

In [47]:
# create map
map_clusters = folium.Map(location=[Toronto_latitude, Toronto_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Neighbourhood Latitude'], toronto_merged['Neighbourhood Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Now we examin each cluster

This is cluster 0

In [50]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,43.73057,-79.310771,Park,0,Grocery Store,Park,Pharmacy,Zoo Exhibit,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant
3,43.73057,-79.312427,Grocery Store,0,Grocery Store,Park,Pharmacy,Zoo Exhibit,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant
4,43.73057,-79.307432,Pharmacy,0,Grocery Store,Park,Pharmacy,Zoo Exhibit,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant
102,43.81139,-79.199056,Fast Food Restaurant,0,Zoo Exhibit,Fast Food Restaurant,Furniture / Home Store,Fish Market,Fish & Chips Shop,Field,Flea Market,Farmers Market,Food,Farm
103,43.81139,-79.200708,Zoo Exhibit,0,Zoo Exhibit,Fast Food Restaurant,Furniture / Home Store,Fish Market,Fish & Chips Shop,Field,Flea Market,Farmers Market,Food,Farm
104,43.81139,-79.199596,Furniture / Home Store,0,Zoo Exhibit,Fast Food Restaurant,Furniture / Home Store,Fish Market,Fish & Chips Shop,Field,Flea Market,Farmers Market,Food,Farm
246,43.78574,-79.156820,Construction & Landscaping,0,Construction & Landscaping,Bar,Farm,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Food & Drink Shop
247,43.78574,-79.163085,Bar,0,Construction & Landscaping,Bar,Farm,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Food & Drink Shop
357,43.69211,-79.428705,Field,0,Hockey Arena,Grocery Store,Field,Trail,Park,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant
358,43.69211,-79.426106,Trail,0,Hockey Arena,Grocery Store,Field,Trail,Park,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant


This is Cluster 1

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
x = 2
x